# NOTE: This notebook was adapted from an existing notebook from HuggingFace (original here: https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/language_modeling_from_scratch.ipynb) by Kelly Peterson for the 2024 DELPHI NLP course at the University of Utah.  It has been changed to:
1. Use a smaller amount of data to allow training a model during a course which is non-optimal.
2. Remove one of the tasks in the original notebook
3. Show masked language model (MDM) predictions

## Also note that you should change the runtime of your notebook to be a TPU or GPU otherwise this could take 10x or 20x longer...  

1. Above, select "Runtime"
2. Then select "Change Runtime Type"
3. In the dialog that opens, select "T4 GPU" or "TPU"
4. Then click OK

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets.

In [1]:
! pip install datasets transformers transformers[torch] pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00


Then you need to install Git-LFS. Uncomment the following instructions:

In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [3]:
import math

import transformers

from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments

import torch

print(transformers.__version__)
print(torch.__version__)

4.35.2
2.1.0+cu121


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

# Train a language model

In this notebook, we'll see how to train a [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling task. We will cover

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to train a model on it.

This notebooks assumes you have trained a tokenizer on the corpus you are using, see the [How to train a tokenizer](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) notebook ([open in colab](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb)).

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [4]:
from datasets import load_dataset

# This is really big... it would take us 30+ minutes to train, even on GPU...
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [5]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [6]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,text
0,"The final two performances , in Tempe , Arizona , on 19 and 20 December 1987 , were filmed for the 1988 film Rattle and Hum . U2 sang the refrain "" el pueblo vencerá "" , which means "" a people united will overcome "" in Spanish , at the conclusion of the song . Bono noted that the Madres use the phrase for motivation . The Edge said "" we 're so close to a Spanish speaking part of the world , we felt that maybe people at the concert might pick up on this lyric . "" Bono added that they had closed every concert since 1983 with the song "" 40 "" , and so they were looking to replace it with "" Mothers of the Disappeared "" from that point on . He explained , "" If the people of Arizona sing this , and if it goes into the film and onto the record , wherever we go in a way for the next few years , that will be taken up again . It 'll be an interesting experiment ... "" . The footage was considered for the closing sequence of the film , but the band eventually decided against including it . "" Pride ( In the Name of Love ) "" was used as the final live song , and "" All I Want Is You "" was chosen to play over the credits . \n"
1,"In the southern portion of its range , the cougar and jaguar share overlapping territory . The jaguar tends to take larger prey and the cougar smaller where they overlap , reducing the cougar 's size and also further reducing the likelihood of direct competition . Of the two felines , the cougar appears best able to exploit a broader prey niche and smaller prey . \n"
2,
3,"The island of Dominica suffered severe damage from Hurricane Omar . The village of Scotts Head , with a population of 450 , was cut off from the rest of the country as roads were extensively damaged . The village also suffered water losses , electricity shortage , and landline telephones were cut off . Ports throughout the country were severely damaged . All barge access for hauling sand and stones were destroyed . The airport also sustained some damage . Seven boats ran aground during the storm . Minor damage was reported in Anguilla . Two hotels sustained roof damage , downed treed knocked down power lines causing scattered power outages , and the rough seas caused severe beach erosion . Three cargo ships and seven boats ran aground and two other boats sank . \n"
4,
5,
6,= = = = High jump = = = = \n
7,
8,= = = Tropical Storm Aka = = = \n
9,= = Reception = = \n


In [9]:
print(type(datasets['train']))


trimmed_train_dataset = datasets['train'].select(range(1000))
trimmed_test_dataset = datasets['test'].select(range(100))

print(trimmed_train_dataset.__len__())
print(trimmed_test_dataset.__len__())

<class 'datasets.arrow_dataset.Dataset'>
1000
100


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Masked language modeling

For masked language modeling (MLM) we are going to preprocess our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens). If you use a tokenizer you trained yourself, make sure the `[MASK]` token is among the special tokens you passed during training!

We will use the [`bert-base-cased`](https://huggingface.co/bert-based-cased) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead. For the tokenizer, replace the checkpoint by the one you trained.

In [10]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [11]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)


tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/175k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/365k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [13]:
tokenized_train_dataset = trimmed_train_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
tokenized_test_dataset = trimmed_test_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [14]:
tokenized_train_dataset[1]

{'input_ids': [2, 33, 9773, 10627, 4171, 33, 3],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

We group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [15]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [16]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [17]:
lm_train_dataset = tokenized_train_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

lm_test_dataset = tokenized_test_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

Next  we use a model suitable for masked LM:

In [18]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

We redefine our `TrainingArguments`:

In [19]:
# we will not push our model to the Internet today... Maybe another day...
push_to_hub = False

training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=push_to_hub,
)

The last arguments are to setup everything so we could theoretically push the model to the [Hub](https://huggingface.co/models) at the end of training. Remove the two of them if you didn't follow the installation steps at the top of the notebook, otherwise you can change the value of `push_to_hub_model_id` to something you would prefer.

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [20]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_train_dataset,
    eval_dataset=lm_test_dataset,
    data_collator=data_collator,
)

# Here we go... time to back this model in the oven... This will take time...

In [22]:
# This will take 1 to 2 minutes when training on GPU..
# Much faster than if we used the full dataset which would take 30+ minutes

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,8.773790
2,No log,8.389771
3,No log,8.254836


TrainOutput(global_step=186, training_loss=8.55461202641969, metrics={'train_runtime': 53.4218, 'train_samples_per_second': 27.461, 'train_steps_per_second': 3.482, 'total_flos': 96528645817344.0, 'train_loss': 8.55461202641969, 'epoch': 3.0})

We can evaluate our model on the validation set. The perplexity is not
great since we used a small fraction of the data

In [23]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 3917.39


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

# Now for some fun experiments... Let's see how well our model fills in the blanks

In [24]:
pred_model = transformers.pipeline('fill-mask', model = model, tokenizer = tokenizer, device = torch.cuda.current_device())

text = "The Milky Way is a [MASK] galaxy."

preds = pred_model(text)

#for pred in preds:
#    print(f">>> {pred['sequence']}")

## Now try your own.  See if the model has any different behavior with context...

In [25]:
# Make sure to include this [MASK] token in your test sentence

your_own_text = "[MASK]"

preds = pred_model(your_own_text)

#for pred in preds:
#    print(f">>> {pred['sequence']}")

TrainOutput(global_step=186, training_loss=8.55461202641969, metrics={'train_runtime': 53.4218, 'train_samples_per_second': 27.461, 'train_steps_per_second': 3.482, 'total_flos': 96528645817344.0, 'train_loss': 8.55461202641969, 'epoch': 3.0})